# Lab 6: Variable Selection and Regularization

## Part I: Different Model Specs
### A. Regression without regularization
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary linear regression

In [30]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, make_scorer, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

In [31]:
hitters = pd.read_csv(r"C:\Users\achur\OneDrive\Desktop\School\CP Fall 2024\544\Hitters.csv")

In [52]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd
import numpy as np

def regression_analysis(model_type, X, y, categorical_columns, interaction_columns, param_grid=None, cv=5):
    """
    Performs regression analysis with linear, ridge, lasso, or elastic net regression using a pipeline.

    Parameters:
    - model_type: String specifying model ('linear', 'ridge', 'lasso', or 'elasticnet')
    - X: DataFrame of predictors
    - y: Series of target values
    - categorical_columns: List of columns to dummify
    - interaction_columns: List of columns to create interactions
    - param_grid: Dictionary of hyperparameters for grid search (for ridge, lasso, or elastic net)
    - cv: Number of cross-validation folds

    Returns:
    - Fitted model, cross-validated MSE, and DataFrame of coefficients (if applicable)
    """

    # colummn transformer
    ct_dummies = ColumnTransformer(
        [("dummify", OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_columns)],
        remainder="passthrough"
    ).set_output(transform="pandas")

    ct_inter = ColumnTransformer(
        [
            ("interaction", PolynomialFeatures(interaction_only=True, include_bias=False), interaction_columns)
        ],
        remainder="drop"
    ).set_output(transform="pandas")

    # model type
    if model_type == 'linear':
        model = LinearRegression()
    elif model_type == 'ridge':
        model = Ridge()
    elif model_type == 'lasso':
        model = Lasso()
    elif model_type == 'elasticnet':
        model = ElasticNet()
    else:
        raise ValueError("Invalid model_type. Choose from 'linear', 'ridge', 'lasso', or 'elasticnet'.")

    # pipeline
    pipeline = Pipeline([
        ('dummification', ct_dummies),
        ('interactions', ct_inter),
        ('scaler', StandardScaler()),
        ('regressor', model)
    ])

    # pipeline and cross-validation for linear
    if model_type == 'linear':
        pipeline.fit(X, y)
        mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
        cv_scores = cross_val_score(pipeline, X, y, cv=cv, scoring=mse_scorer)
        average_mse = -np.mean(cv_scores)

        # Extract coefficients if applicable
        regressor = pipeline.named_steps['regressor']
        coefficients = regressor.coef_
        transformed_features = pipeline.named_steps['interactions'].get_feature_names_out()
        coeff_df = pd.DataFrame({'Feature': transformed_features, 'Coefficient': coefficients})
        coeff_df = coeff_df.sort_values(by='Coefficient', key=abs, ascending=False)

        print(f'Linear Regression - Estimated MSE: {average_mse:.2f}')
        print('Most important coefficients:')
        print(coeff_df.head())
        return pipeline, average_mse, coeff_df

    # hyperparamter tuning for ridge, lasso, and elasticnet
    if model_type in ['ridge', 'lasso', 'elasticnet']:
        if not param_grid:
            raise ValueError("param_grid must be provided for ridge, lasso, and elastic net regression.")

        grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='neg_mean_squared_error')
        grid_search.fit(X, y)

        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
        best_mse = -grid_search.best_score_

        # coefficients
        regressor = best_model.named_steps['regressor']
        coefficients = regressor.coef_
        transformed_features = best_model.named_steps['interactions'].get_feature_names_out()
        coeff_df = pd.DataFrame({'Feature': transformed_features, 'Coefficient': coefficients})
        coeff_df = coeff_df.sort_values(by='Coefficient', key=abs, ascending=False)

        print(f'{model_type.capitalize()} Regression - Best Parameters: {best_params}')
        print(f'Estimated MSE: {best_mse:.2f}')
        print('Most important coefficients:')
        print(coeff_df.head())

        return best_model, best_mse, coeff_df


2. Fit this pipeline to the full dataset, and interpret a few of the most important coefficients.
3. Use cross-validation to estimate the MSE you would expect if you used this pipeline to predict 1989 salaries

In [48]:
X = hitters.drop(columns=['Salary'])
y = hitters['Salary'].dropna()
X = X.loc[y.index]

categorical_columns = ['League']
interaction_columns = ["remainder__Years", "dummify__League_A"]

# use function for linear regression
regression_analysis('linear', X, y, categorical_columns, interaction_columns)

Linear Regression - Estimated MSE: 174559.50
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   166.978056
2  interaction__remainder__Years dummify__League_A    27.099368
1                   interaction__dummify__League_A   -18.891896
Linear Regression - Estimated MSE: 174559.50
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   166.978056
2  interaction__remainder__Years dummify__League_A    27.099368
1                   interaction__dummify__League_A   -18.891896


(Pipeline(steps=[('dummification',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('dummify',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse_output=False),
                                                   ['League'])])),
                 ('interactions',
                  ColumnTransformer(transformers=[('interaction',
                                                   PolynomialFeatures(include_bias=False,
                                                                      interaction_only=True),
                                                   ['remainder__Years',
                                                    'dummify__League_A'])])),
                 ('scaler', StandardScaler()),
                 ('regressor', LinearRegression())]),
 174559.4960037865,
                                           

Interpretation: The first feature represents the linear relationship between Years and Salary. This means that on average, for each additional year of playing, the predicted Salary increases by 166.978k, holding all other variables constant. The second feature represnts the linear relationship between the interaction terms Year and League A. This means that for players in League A, each additional year of playing, the salary is expected to increase by 27.099k. The third feature represents the relationship between League A and the other league. This means that being in League A is associated with a -18.89k decrease in salary compared to the other league

### B. Ridge regression
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda hyperparameter.
3. Fit the pipeline with your chosen lambda to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [49]:
# use function for ridge with hyper parameter
param_grid_ridge = {'regressor__alpha': [0.1, 1.0, 10.0, 100.0]}
regression_analysis('ridge', X, y, categorical_columns, interaction_columns, param_grid=param_grid_ridge)

Ridge Regression - Best Parameters: {'regressor__alpha': 10.0}
Estimated MSE: 174434.04
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   158.320468
2  interaction__remainder__Years dummify__League_A    32.363761
1                   interaction__dummify__League_A   -21.636648
Ridge Regression - Best Parameters: {'regressor__alpha': 10.0}
Estimated MSE: 174434.04
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   158.320468
2  interaction__remainder__Years dummify__League_A    32.363761
1                   interaction__dummify__League_A   -21.636648


(Pipeline(steps=[('dummification',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('dummify',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse_output=False),
                                                   ['League'])])),
                 ('interactions',
                  ColumnTransformer(transformers=[('interaction',
                                                   PolynomialFeatures(include_bias=False,
                                                                      interaction_only=True),
                                                   ['remainder__Years',
                                                    'dummify__League_A'])])),
                 ('scaler', StandardScaler()),
                 ('regressor', Ridge(alpha=10.0))]),
 174434.03772695793,
                                           

### C. Lasso Regression
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda hyperparameter.
3. Fit the pipeline with your chosen lambda to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [50]:
# use function for lasso with hyperparameter
param_grid_lasso = {'regressor__alpha': [0.01, 0.1, 1.0, 10.0]}
regression_analysis('lasso', X, y, categorical_columns, interaction_columns, param_grid=param_grid_lasso)

Lasso Regression - Best Parameters: {'regressor__alpha': 10.0}
Estimated MSE: 173695.16
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   169.341332
2  interaction__remainder__Years dummify__League_A     2.050551
1                   interaction__dummify__League_A    -0.000000
Lasso Regression - Best Parameters: {'regressor__alpha': 10.0}
Estimated MSE: 173695.16
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   169.341332
2  interaction__remainder__Years dummify__League_A     2.050551
1                   interaction__dummify__League_A    -0.000000


(Pipeline(steps=[('dummification',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('dummify',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse_output=False),
                                                   ['League'])])),
                 ('interactions',
                  ColumnTransformer(transformers=[('interaction',
                                                   PolynomialFeatures(include_bias=False,
                                                                      interaction_only=True),
                                                   ['remainder__Years',
                                                    'dummify__League_A'])])),
                 ('scaler', StandardScaler()),
                 ('regressor', Lasso(alpha=10.0))]),
 173695.16435016156,
                                           

### D. Elastic Net
1. Create a pipeline that includes all the columns as predictors for Salary, and performs ordinary ridge regression.
2. Use cross-validation to tune the lambda and alpha hyperparameters.
3. Fit the pipeline with your chosen hyperparameters to the full dataset, and interpret a few of the most important coefficients.
4. Report the MSE you would expect if you used this pipeline to predict 1989 salaries.

In [51]:
# use function for elasticnet with hyperparameter
param_grid_elasticnet = {'regressor__alpha': [0.01, 0.1, 1.0, 10.0], 'regressor__l1_ratio': [0.2, 0.5, 0.8]}
regression_analysis('elasticnet', X, y, categorical_columns, interaction_columns, param_grid=param_grid_elasticnet)

Elasticnet Regression - Best Parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.5}
Estimated MSE: 174433.42
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   156.039981
2  interaction__remainder__Years dummify__League_A    33.245256
1                   interaction__dummify__League_A   -21.884569
Elasticnet Regression - Best Parameters: {'regressor__alpha': 0.1, 'regressor__l1_ratio': 0.5}
Estimated MSE: 174433.42
Most important coefficients:
                                           Feature  Coefficient
0                    interaction__remainder__Years   156.039981
2  interaction__remainder__Years dummify__League_A    33.245256
1                   interaction__dummify__League_A   -21.884569


(Pipeline(steps=[('dummification',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('dummify',
                                                   OneHotEncoder(handle_unknown='ignore',
                                                                 sparse_output=False),
                                                   ['League'])])),
                 ('interactions',
                  ColumnTransformer(transformers=[('interaction',
                                                   PolynomialFeatures(include_bias=False,
                                                                      interaction_only=True),
                                                   ['remainder__Years',
                                                    'dummify__League_A'])])),
                 ('scaler', StandardScaler()),
                 ('regressor', ElasticNet(alpha=0.1))]),
 174433.42238269112,
                                       

## Part II. Variable Selection
Based on the above results, decide on:

* Which numeric variable is most important.

* Which five numeric variables are most important

* Which categorical variable is most important

For each of the four model specifications, compare the following possible feature sets:

1. Using only the one best numeric variable.

2. Using only the five best variables.

3. Using the five best numeric variables and their interactions with the one best categorical variable.

Report which combination of features and model performed best, based on the validation metric of MSE. (Note: lambda and alpha must be re-tuned for each feature set.)

## Part III. Discussion

### A. Ridge

Compare your Ridge models with your ordinary regression models. How did your coefficients compare? Why does this make sense?

### B. Lasso

Compare your LASSO model in I with your three LASSO models in II. Did you get the same lambda results? Why does this make sense? Did you get the same MSEs? Why does this make sense?

### C. Elastic Net

Compare your MSEs for the Elastic Net models with those for the Ridge and LASSO models. Why does it make sense that Elastic Net always “wins”?

## Part IV: Final Model

Fit your final best pipeline on the full dataset, and summarize your results in a few short sentences and a plot.

## Appendix and References